In [1]:
!pip install langchain openai langchain-openai

You should consider upgrading via the 'D:\Code\PromptEngineering_Langchain\env\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=os.getenv("OPENAI_API_KEY"), temperature=1.5)

In [4]:
llm.invoke("how can langsmith help with testing?")

AIMessage(content='Langsmith, as a language services provider, can assist with testing by providing translation and localization services for different language versions of the software or product being tested. This can help ensure that the product works properly in different languages and dialects. Langsmith can also provide linguistic validation and cultural adaptation testing to ensure the product is appropriate and effective for specific target audiences. Additionally, Langsmith can support user acceptance testing by providing multilingual testers who can test the product in different languages and provide feedback on its usability and functionality.')

In [5]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

In [6]:
chain = prompt | llm 

In [7]:
chain.invoke({"input": "how can langsmith help with testing?"})

AIMessage(content="LangSmith can greatly help with testing by providing a controlled environment for developers and QA testers to run automated tests with dynamically generated content. This allows for comprehensive testing coverage without the need for manual data entry or setup.\n\nAdditionally, LangSmith can be used to quickly set up localized testing environments, reducing the testing time required for each locale. This is especially useful when testing language-specific features or functionality in an application.\n\nUsing LangSmith's powerful API, testers can integrate automated tests directly into their existing testing frameworks and pipelines, streamlining the testing process and ensuring efficient and reliable testing.\n\nOverall, LangSmith simplifies testing efforts, accelerates testing cycles, and improves overall testing accuracy by providing a versatile platform for automated test data generation and localization testing.")

In [8]:
# The output of a ChatModel (and therefore, of this chain) is a message. 
# However, it's often much more convenient to work with strings. Let's add a simple output parser to convert the chat message to a string.
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [9]:
chain = prompt | llm | output_parser

In [10]:
chain.invoke({"input": "how can langsmith help with testing?"})

'Langsmith is a powerful tool that can greatly assist in the testing process by enabling language analysis. Some ways in which Langsmith can help with testing include:\n\n1. Detecting language variants: Langsmith can identify different language variants, dialects, and styles, helping in creating test datasets that are representative of diverse language usage.\n\n2. Identifying translation errors: Langsmith can spot inaccuracies in translations, flagging any mismatches in syntax or semantics that could impact the quality of an application or content.\n\n3. Checking text coherence: Langsmith can verify the coherence and logical consistency of text, ensuring that user interfaces and content set for testing are clear and comprehensible.\n\n4. Automating linguistic testing tasks: Langsmith can automate certain linguistic testing tasks, saving time and effort for QA teams while ensuring thorough language analysis for validation.\n\n5. Enhancing internationalization testing: Langsmith can pro

In [11]:
# Basic Example of a Prompt.
response = llm.invoke("What are the 7 wonders of the world?")
print(f"Response: {response}")

Response: content='The Seven Wonders of World are:\n\n1. Great Wall of China\n2. Petra, Jordan\n3. The Colosseum, Italy\n4. Machu Picchu, Peru\n5. Chichen Itza, Mexico\n6. Taj Mahal, India\n7. Christ the Redeemer, Brazil'


In [12]:
# Basic Example of a Prompt Template.
prompt_template = ChatPromptTemplate.from_template(
    "List {n} cooking recipe ideas {cuisine} cuisine (name only)."
)
prompt = prompt_template.format(n=3, cuisine="italian")
print(f"Templated Prompt: {prompt}")

response = llm.invoke(prompt)
print(f"Response: {response}")

Templated Prompt: Human: List 3 cooking recipe ideas italian cuisine (name only).
Response: content='1. Spaghetti aglio e olio\n2. Margherita pizza\n3. tiramisu'


# Response Parser
- formatting responses
- structured output

ex: Movie
- title: str
- genre: list[str]
- year: int

In [13]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

class Movie(BaseModel):
    title: str = Field(description="movie title")
    genre: list[str] = Field(description="movie genre")
    year: int = Field(description="year of relase")
    
parser = PydanticOutputParser(pydantic_object=Movie)

In [14]:

prompt_template_text = """
Response with a movie recommendation based on the query:\n
{format_instructions}\n
{query}
"""

format_instructions = parser.get_format_instructions()
print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"title": {"title": "Title", "description": "movie title", "type": "string"}, "genre": {"title": "Genre", "description": "movie genre", "type": "array", "items": {"type": "string"}}, "year": {"title": "Year", "description": "year of relase", "type": "integer"}}, "required": ["title", "genre", "year"]}
```


In [15]:
prompt_template = ChatPromptTemplate.from_template(
    prompt_template_text
)

prompt = prompt_template.format(query="A 90s movie with Nicolas Cage.", format_instructions=format_instructions)
text_output = llm.predict(prompt)
print(text_output)

d:\Code\PromptEngineering_Langchain\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


``` 
{
  "title": "Con Air",
  "genre": ["Action", "Crime", "Thriller"],
  "year": 1997
}
```


In [16]:
parsed_output = parser.parse(str(text_output))
print(parsed_output)

title='Con Air' genre=['Action', 'Crime', 'Thriller'] year=1997


In [17]:
# Using LCEL
chain = prompt_template | llm | parser
response = chain.invoke({"query": "A 90s movie with Nicolas Cage.", "format_instructions": format_instructions})
print(response)

title='Face/Off' genre=['Action', 'Crime', 'Sci-Fi'] year=1997


# LLM Chains
 Chaining prompts, input of a prompt depends on the output of another prompt

In [18]:
prompt_1 = """
Come up with a short plot synopsis summary (2-3 lines) for a {genre} movie.
"""

prompt_2 = """
Suggest 5 movie title ideas for this movie: \n\n {synopsis}
"""

In [19]:
from langchain.chains import LLMChain, SequentialChain

p1_template = ChatPromptTemplate.from_template(prompt_1)
chain_1 = LLMChain(llm=llm, prompt=p1_template, output_key="synopsis")

p2_template = ChatPromptTemplate.from_template(prompt_2)
chain_2 = LLMChain(llm=llm, prompt=p2_template, output_key="titles")

In [20]:
complex_chain = SequentialChain(
    chains=[chain_1, chain_2],
    input_variables=["genre"],
    output_variables=["synopsis", "titles"],
    verbose=True,
)
output = complex_chain({"genre": "comedy"})
print(f"Output: {output}")

d:\Code\PromptEngineering_Langchain\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...

> Finished chain.
Output: {'genre': 'comedy', 'synopsis': 'Two friends launch a scheme to make it big by selling exotic seaweed, but when their product turns out to be hazardous instead after endless mishaps and mistaken identity, they must scramble to maintain their friendship and reputation.', 'titles': '1. "Tangled Seaweed Scandal" \n2. "Salty Business: The Seaweed Saga" \n3. "Into the Deep: A Seaweed Salesmanship Story" \n4. "Seaweed Mishaps and Mischief" \n5. "Underwater Entrepreneurs: The Seaweed Fiasco"'}


# AGENT

In [21]:
!pip install numexpr google-search-results

You should consider upgrading via the 'D:\Code\PromptEngineering_Langchain\env\Scripts\python.exe -m pip install --upgrade pip' command.


In [22]:
from langchain.chains import LLMMathChain
# Chain that interprets a prompt and executes python code to do math.
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)

In [23]:
# google search api
from langchain.utilities import SerpAPIWrapper
os.environ["SERPAPI_API_KEY"] = os.getenv("SERP_API_KEY")
search = SerpAPIWrapper()

## Tools
Tools are interfaces that an agent can use to interact with the world.
They combine a few things:

- The name of the tool
- A description of what the tool is
- JSON schema of what the inputs to the tool are
- The function to call
- Whether the result of a tool should be returned directly to the user

In [24]:
from langchain.agents import Tool
tools = [
    Tool(
        name="search",
        description="Search for information Google.",
        func=search.run,
    ),
    Tool(
        name="calculator",
        description="Use this tool to calculate the difference between numbers.",
        func=llm_math_chain.run,
    ),
]

In [25]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
# The MessagesPlaceholder serves as a dynamic content placeholder within the chat prompt template. 
# It's designed to be replaced by actual content at runtime. 
# The variable_name="agent_scratchpad" parameter specifies the name of the variable that will be replaced with the actual content generated 
# or used by the agent during its execution.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [26]:
from langchain_core.utils.function_calling import convert_to_openai_function

from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.agents import AgentExecutor

# Convert a raw function/class to an OpenAI function.
llm_with_tools = llm.bind(functions=[convert_to_openai_function(t) for t in tools])

# Define the agent schema.
agent_schema = {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_to_openai_functions(x["intermediate_steps"]),
}
# format to openai functions converts (AgentAction, tool output) tuples into FunctionMessages.
# Returns list of messages to send to the LLM for the next prediction

agent = agent_schema | prompt | llm_with_tools | OpenAIFunctionsAgentOutputParser()

# Create an agent executor. Agent that is using tools.
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": "What is the population difference between 🇺🇸 and 🇬🇧?"})



> Entering new AgentExecutor chain...

Invoking: `search` with `population of usa`


{'type': 'population_result', 'place': 'United States', 'population': '331.9 million', 'year': '2021'}
Invoking: `search` with `population of uk`


{'type': 'population_result', 'place': 'United Kingdom', 'population': '67.33 million', 'year': '2021'}
Invoking: `calculator` with `331900000-67330000`




> Entering new LLMMathChain chain...
331900000-67330000```text
331900000-67330000
```
...numexpr.evaluate("331900000-67330000")...

Answer: 264570000
> Finished chain.
Answer: 264570000The population difference between the United States 🇺🇸 and the United Kingdom 🇬🇧 is approximately 264,570,000.

> Finished chain.


{'input': 'What is the population difference between 🇺🇸 and 🇬🇧?',
 'output': 'The population difference between the United States 🇺🇸 and the United Kingdom 🇬🇧 is approximately 264,570,000.'}

In [ ]:
# See COHERE API